DSA5206 Project Part 1

In [1]:
import numpy as np
import random
random.seed(5206)

ns, nt, dt = 1000, 1000, 1e-3
A = np.array([[-1, 0], [0, -10]])

# Simulate trajectories
def sim_traj(A, nt, dt):
    h = np.zeros((2, nt))
    for t in range(1, nt):
        h[:, t] = h[:, t-1] + (A @ h[:, t-1] + np.array([1e-3, -1e3]) * np.random.normal()) * dt
    return h

trajs = np.array([sim_traj(A, nt, dt) for i in range(ns)])

# Calculate leading POD mode
data = np.mean(trajs, axis=0)
C = data @ data.T / nt
vals, vecs = np.linalg.eig(C)
mode = vecs[:, np.argmax(vals)]
print(mode)


[ 1.7255986e-06 -1.0000000e+00]


In [4]:
data.shape

(2, 1000)

In [5]:
from scipy.linalg import expm
from scipy.integrate import quad

# Define the matrices A and B
A = np.array([[-1, 0], [0, -10]])
B = np.array([[1e-3], [1e3]])

# Function to calculate the matrix exponential product at each point in time
def integrand(t, A, B):
    return expm(A * t) @ B @ B.T @ expm(A.T * t)

# Integration bounds
t0 = 0
t1 = 1

# Perform the integration for each element of the Gramian matrix
Wc = np.zeros((2, 2))
for i in range(2):
    for j in range(2):
        # Using a lambda function to pass i and j along with the integrand
        Wc[i, j], _ = quad(lambda t: integrand(t, A, B)[i, j], t0, t1)

# Compute the eigenvalues of the Gramian
eigenvalues = np.linalg.eigvals(Wc)

print("Controllability Gramian Wc:", Wc)
print("Eigenvalues of Wc:", eigenvalues)

Controllability Gramian Wc: [[4.32332358e-07 9.09075726e-02]
 [9.09075726e-02 4.99999999e+04]]
Eigenvalues of Wc: [2.67049472e-07 4.99999999e+04]


In [6]:
# Define the transformation matrix T
T = np.array([[10**3, 0], [0, 10**-3]])

# Calculate the inverse of T
T_inv = np.linalg.inv(T)

# Compute the new A and B matrices for the transformed system
A_tilde = T_inv @ A @ T
B_tilde = T_inv @ B

A_tilde, B_tilde

(array([[ -1.,   0.],
        [  0., -10.]]),
 array([[1.e-06],
        [1.e+06]]))

In [ ]:
# Calculate the controllability Gramian for the transformed system
Wc_tilde = np.zeros((2, 2))
for i in range(2):
    for j in range(2):
        Wc_tilde[i, j], _ = quad(lambda t: integrand(t, A_tilde, B_tilde)[i, j], t0, t1)

# Compute the eigenvalues of Wc_tilde(1)
eigenvalues_tilde = np.linalg.eigvals(Wc_tilde)

Wc_tilde, eigenvalues_tilde